# 準備
Google ドライブをマウントし，（必要なら）Graphillionをインストールする．また，タイムアウトを設定するために`timeout-decorator`をインストールする．以下のセルはノートブックを開くたびに実行する必要がある．

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install timeout-decorator
!pip install graphillion

Mounted at /content/drive
  Preparing metadata (setup.py) ... done
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5005 sha256=1d410f4041ff0b45a3099fe9e4e99ffe07d0e09ebf05b5c91f6510baf46cdbe1
  Stored in directory: /root/.cache/pip/wheels/68/2f/bc/76f1192d474666d41ae6f09813fccbd00fe3f07e8261c4cff5
Successfully built timeout-decorator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for graphillion: filename=Graphillion-1.8-cp310-cp310-linux_x86_64.whl size=6957515 sha256=79699e353806a2927ca0491dbe9bb96eaab479878a133f931a6e6178a2d69519
  Stored in directory: /root/.cache/pip/wheels/2f/c5/d4/380d0c8b37ccd22ec2b4ecc0b3bcc2731180514af59d8f3623
Successfully built graphillion


タイムアウトを設定するために，`timeout_decorator`モジュールを用いる．タイムアウトを設定したい関数の前に`@timeout_decorator.timeout(sec)`と記述すると，`sec`秒のタイムアウトを設定できる．関数を実行してから`sec`秒経つと`timeout_decorator.TimeoutError`が送出される．そのままだと実行が止まってしまうので，今回は次のデータセットに進むために`try-except`文で処理する．

- 参考: [Timeout decorator](https://github.com/pnpnpn/timeout-decorator)

In [ ]:
# import time
# import timeout_decorator

# @timeout_decorator.timeout(5)
# def mytest():
#     try:
#         print("Start")
#         for i in range(1,10):
#             time.sleep(1)
#             print("{} seconds have passed".format(i))
#     except timeout_decorator.TimeoutError:
#         print('Timed out')
#         return

# mytest()

# 実験

以下のコードは，すべてのベンチマークに対して演習で扱った前処理＋バックトラックを実行する．
タイムアウトは1ケースあたり1分としている．

## 注意

- 実行に時間がかかる（単純計算で1分×50ケース=50分）
- Graphillionを用いる場合，メモリ不足になるとループが止まってしまう


In [18]:
import networkx as nx
import os
import time
import timeout_decorator
import collections
import itertools
import copy
import heapq
import numpy as np
import matplotlib.pyplot as plt
from graphillion import GraphSet as gs

def build_greedy_universe(universe, source, neighbors):
    sorted_edges = []
    indexed_edges = set()
    gs._vertices = set()
    gs._weights = {}

    for e in universe:
        sorted_edges.append(e[:2])
        indexed_edges.add(e[:2])

    sorted_edges = greedy_sort(indexed_edges, source,neighbors)
    return sorted_edges

def greedy_sort(indexed_edges, source,neighbors):
    vertices = set(neighbors.keys())
    sorted_edges = []
    visited_vertices = set()
    u = source

    degree = dict()
    for v in vertices:
        degree[v] = len(neighbors[v])

    heap = []
    while True:
        visited_vertices.add(u)
        for v in sorted(neighbors[u]):
            degree[v] -= 1
            if v in visited_vertices:
                degree[u] -= 1
                e = (u, v) if (u, v) in indexed_edges else (v, u)
                sorted_edges.append(e)
                if degree[v]:
                    for w in sorted(neighbors[v]):
                        if w not in visited_vertices:
                            heapq.heappush(heap, (degree[v], degree[w], w))
        for v in sorted(neighbors[u]):
            if v not in visited_vertices:
                  heapq.heappush(heap, (degree[u], degree[v], v))
        if visited_vertices == vertices:
            break
        while u in visited_vertices:
            if not heap:
                u = min(vertices - visited_vertices)
            else:
                u = heapq.heappop(heap)[2]
    assert set(indexed_edges) == set(sorted_edges)
    return sorted_edges

def get_all_neighbors(G: nx.Graph):
    all_neighbors = {}
    for i in G.nodes:
        all_neighbors[i] = set(G.neighbors(i))
    return all_neighbors

def frontier_size(
    edge,
    processed_nodes: set[int],
    current_frontier: set[int],
    current_neighbors: dict[int, list[int]],
):
    current_frontier |= set(edge)
    for i, v in enumerate(edge):
        current_neighbors[v].remove(edge[1 - i])
        if len(current_neighbors[v]) == 0:
            processed_nodes.add(v)
    current_frontier -= processed_nodes
    return len(current_frontier)


def all_frontier_size(edge_order, all_neighbors):
    score = 0
    current_neighbors: dict[int, list[int]] = copy.deepcopy(all_neighbors)
    processed_nodes = set()
    current_frontier = set()
    for edge in edge_order:
        score += (
            frontier_size(edge, processed_nodes, current_frontier, current_neighbors)
            ** 2
        )
    return score

def get_path_distance(G, v, s, t):
    distance_s = nx.shortest_path_length(G, source=v, target=s)
    distance_t = nx.shortest_path_length(G, source=v, target=t)
    sum_distance = distance_s + distance_t
    diff_distance = abs(distance_s - distance_t)
    return (sum_distance, diff_distance)

def get_edge_order(G, s, t):
    all_neighbors = get_all_neighbors(G)
    scores = []
    edge_orders = []
    i_to_v = {}

    for i, v in enumerate(G.nodes):
        edge_order = build_greedy_universe(G.edges, source=v,neighbors=all_neighbors)
        score = all_frontier_size(
            edge_order,
            all_neighbors,
        )
        scores.append(score)
        edge_orders.append(edge_order)
        i_to_v[i] = v

    indices = np.argsort(scores)[: len(scores) // 4]
    if len(indices) == 0:
        return edge_orders[0]
    # sorted_score = sorted(scores)[: len(scores) // 4]
    path_distances = [get_path_distance(G, i_to_v[i], s, t) for i in indices]
    distance_indices = [
        i for i, _ in sorted(enumerate(path_distances), key=lambda x: x[1])
    ]

    sorted_edge_order = edge_orders[indices[distance_indices[0]]]
    return sorted_edge_order

def remove_far_vertices(G,s,t,l):
    dict_s = nx.shortest_path_length(G, source=s)
    dict_t = nx.shortest_path_length(G, source=t)
    far_vertices = [v for v in G.nodes if dict_s[v]+dict_t[v] > l]
    prob.G.remove_nodes_from(far_vertices)

def swap_edges(edge_order, i, perm):
    n = len(edge_order)
    if n < 3:
        return edge_order
    new_order = copy.deepcopy(edge_order)

    indices = [i, (i + 1) % n, (i + 2) % n]
    permuted_edges = [edge_order[indices[p]] for p in perm]

    for idx, new_edge in zip(indices, permuted_edges):
        new_order[idx] = new_edge
    return new_order

def local_search(G, edge_order, seconds):
    start_time = time.time()
    all_neighbors = get_all_neighbors(G)
    best_order = copy.deepcopy(edge_order)
    best_score = all_frontier_size(best_order, all_neighbors)

    improved = True
    while improved and time.time() - start_time < seconds:
        improved = False
        for i in range(len(edge_order)):
            for perm in itertools.permutations([0, 1, 2]):
                new_order = swap_edges(best_order, i, perm)
                new_score = all_frontier_size(new_order, all_neighbors)
                if new_score < best_score:
                    best_order = new_order
                    best_score = new_score
                    improved = True
    return best_order

def remove_redundant_blocks(G: nx.Graph, s, t):
    biconn_comps = list(nx.biconnected_components(G))

    # aps = set(nx.articulation_points(G))
    aps = set(
        [
            ap
            for ap, count in collections.Counter(
                itertools.chain.from_iterable(biconn_comps)
            ).items()
            if count > 1
        ]
    )

    bct = []
    for b_i, block in enumerate(biconn_comps):
        for ap in aps.intersection(block):
            bct.append((b_i, -ap))
        if s in block:
            s_bct = b_i
        if t in block:
            t_bct = b_i

    if s_bct == t_bct:
        remove_nodes = set(G.nodes()) - biconn_comps[s_bct]
        G.remove_nodes_from(remove_nodes)
        return

    G_bct = nx.Graph(bct)
    # nx.draw(G_bct, with_labels=True)
    # plt.show()

    inpath_bct_nodes = nx.bidirectional_shortest_path(G_bct, s_bct, t_bct)
    inpath_vertices = set()
    for node in inpath_bct_nodes:
        if node < 0:
            inpath_vertices.add(-node)
        else:
            inpath_vertices |= biconn_comps[node]
    remove_nodes = set(G.nodes()) - inpath_vertices
    G.remove_nodes_from(remove_nodes)


class BT:
    def __init__(self, prob):
        self.prob = prob
        self.shortest_path_lengths = {}
        self.calc_shortest_path_length()

    def calc_shortest_path_length(self):
        for node in self.prob.G.nodes:
            self.shortest_path_lengths[node] = nx.shortest_path_length(
                self.prob.G, source=node, target=self.prob.t
            )

    def dfs(self, v):
        self.call_count += 1
        self.used[v] = True
        self.length += 1

        if v == self.prob.t and self.length <= self.prob.l:
            self.num_path += 1

        else:  # v != t
            for u in self.prob.G.neighbors(v):  # u は v と隣接する頂点
                if not self.used[u]:
                    if self.length + self.shortest_path_lengths[u] > self.prob.l:
                        self.used[u] = False
                        continue

                    self.dfs(u)  # uが探索中のパスに含まれない頂点なら探索

        self.used[v] = False
        self.length -= 1

    def run(self):
        self.call_count = 0
        self.used = {v: False for v in self.prob.G.nodes}
        self.length = 0
        self.num_path = 0

        self.dfs(self.prob.s)


class countPathGS:
    def __init__(self, prob):
        self.prob = prob

    def path_count(self,edge_order, s, t, l):
        gs.set_universe(edge_order, traversal="as-is")
        self.paths = gs.paths(s, t).smaller(l + 1)
        return self.paths.len()

    # 前処理を実行する関数
    def preprocess(self):
        n_pre, m_pre = self.prob.n, self.prob.m

        remove_far_vertices(self.prob.G, self.prob.s, self.prob.t,self.prob.l)
        remove_redundant_blocks(self.prob.G, self.prob.s, self.prob.t)

        self.prob.n = self.prob.G.number_of_nodes()
        self.prob.m = self.prob.G.number_of_edges()
        print(f"preprocessed: (n, m) = ({n_pre}, {m_pre}) -> ({self.prob.n}, {self.prob.m})")

    # 数え上げを実行し，パスの個数を返す
    def run(self):
        # 前処理
        self.preprocess()

        if self.prob.n+self.prob.m < 200:
            self.bt = BT(self.prob)
            self.bt.run()
            return self.bt.num_path

        edge_order = get_edge_order(self.prob.G, self.prob.s, self.prob.t)
        # edge_order = local_search(self.prob.G, edge_order, 60)
        # ローカルサーチは利用しない

        self.score=all_frontier_size(edge_order, get_all_neighbors(self.prob.G))
        if self.score < 18500:
            self.num_path = self.path_count(edge_order, self.prob.s, self.prob.t, self.prob.l)
            return self.num_path
        else:
            self.bt = BT(self.prob)
            self.bt.run()
            return self.bt.num_path


class Problem:
    def __init__(self, file_path):
        self.G = nx.Graph()

        with open(file_path) as f:
            for line in f:
                data = line.split()
                if data[0] == 'c':
                    continue
                elif data[0] == 'p':
                    self.n, self.m = int(data[2]), int(data[3])
                elif data[0] == 'e':
                    if data[1] == data[2]:
                        continue
                    self.G.add_edge(int(data[1]), int(data[2]))
                elif data[0] == 'l':
                    self.l = int(data[1])
                else: # data[0] == 't'
                    self.s, self.t = int(data[1]), int(data[2])

    # メンバを表示する関数
    def show(self):
        print("n = {}, m = {}, l = {}, s = {}, t = {}".format(self.n, self.m, self.l, self.s, self.t))
        print(self.G.edges)


@timeout_decorator.timeout(60)
def count_path(algo):
    try:
        num_path = algo.run()
        return num_path
    except timeout_decorator.TimeoutError:
        return None


# if __name__ == "__main__":
dir_path = "drive/MyDrive/Colab Notebooks/icgca2024"
file_names = os.listdir(dir_path)
successed = [False]*len(file_names)
zdd = {}
calls = {}
scores={}
paths={}
for f_num,file_name in enumerate(file_names):
    if file_name.endswith(".col"):

        file_path = dir_path + "/" + file_name
        prob = Problem(file_path)
        algo = countPathGS(prob)
        print(f"{file_name}")

        %time num_path = count_path(algo) # 実行

        if num_path is not None:
            successed[f_num] = True
            paths[f_num]=num_path

        if algo.prob.n+algo.prob.m < 200:
            calls[f_num]=algo.bt.call_count
        else:
            scores[f_num]=algo.score
            if successed[f_num]:
                zdd[f_num]=len(algo.paths.dumps().split("\n"))

        print(f"#path = {num_path}")
        print()
print(f"solved = {sum(successed)}")

rocketfuel-k_o-10.col
preprocessed: (n, m) = (11, 12) -> (8, 9)
CPU times: user 1.15 ms, sys: 3 µs, total: 1.16 ms
Wall time: 1.15 ms
#path = 4

stanford-feather-lastfm-social-48-49.col
preprocessed: (n, m) = (49, 48) -> (13, 12)
CPU times: user 995 µs, sys: 0 ns, total: 995 µs
Wall time: 988 µs
#path = 1

rsndlib--36.col
preprocessed: (n, m) = (100, 154) -> (90, 144)
CPU times: user 59.3 s, sys: 130 ms, total: 59.4 s
Wall time: 1min
#path = None

rocketfuel-k_o-99.col
preprocessed: (n, m) = (240, 404) -> (136, 298)
CPU times: user 59.3 s, sys: 138 ms, total: 59.5 s
Wall time: 1min
#path = None

stanford-loc-Brightkite-9-10.col
preprocessed: (n, m) = (10, 9) -> (4, 3)
CPU times: user 551 µs, sys: 0 ns, total: 551 µs
Wall time: 542 µs
#path = 1

stanford-musae-wiki_crocodile-1311-1200.col
preprocessed: (n, m) = (1221, 1311) -> (245, 333)
CPU times: user 59.1 s, sys: 175 ms, total: 59.3 s
Wall time: 1min
#path = None

matpower-case145-105.col
preprocessed: (n, m) = (145, 422) -> (131, 40

In [23]:
import pandas as pd
import json

def max_bicc_size(G):
    return max(len(bicc) for bicc in nx.biconnected_components(G))

info_json = {}

for f_num,file_name in enumerate(file_names):
    if file_name.endswith(".col"):
        file_path = dir_path + "/" + file_name
        edges = []
        with open(file_path) as f:
            info = {}
            info['solved'] = str(successed[f_num])
            if successed[f_num]:
                info['paths'] = str(paths[f_num])
            for line in f:
                data = line.split()
                if data[0] == 'c':
                    pass
                elif data[0] == 'p':
                    info['n'] = int(data[2])
                    info['m'] = int(data[3])

                    info['2m/n'] = round(2*info['m'] / info['n'],2)
                elif data[0] == 'e':
                    if data[1] == data[2]:
                        continue
                    edges.append((int(data[1]), int(data[2])))
                elif data[0] == 'l':
                    info['l'] = int(data[1])
                elif data[0] == 't':
                    info['s'] = int(data[1])
                    info['t'] = int(data[2])

            G = nx.Graph(edges)
            info['max_bicc'] = max_bicc_size(G)
            info['maxbi_occ'] = round(100*info['max_bicc'] / info['n'],2)
            info['density'] = round(100*2*info['m'] / (info['n'] * (info['n'] - 1)),2)
            info['cycles'] = len(list(nx.cycle_basis(G)))
            info['cycle_pcts'] = round(100*info['cycles'] / info['n'],2)
            info['max_deg'] = int(np.max(G.degree,axis=0)[1])
            info['l/short'] = round(info['l'] / nx.shortest_path_length(G, source=info['s'], target=info['t']),2)
            if calls.get(f_num) is not None:
                info['calls'] = calls[f_num]
            if zdd.get(f_num) is not None:
                info['zdd'] = zdd[f_num]
            if scores.get(f_num) is not None:
                info['score'] = scores[f_num]

            info_json[file_name] = info


with open("drive/MyDrive/Colab Notebooks/icgca2024/information.json", 'w') as f:
    json.dump(info_json, f)

df = pd.read_json("drive/MyDrive/Colab Notebooks/icgca2024/information.json").transpose()
df.to_excel("drive/MyDrive/Colab Notebooks/icgca2024/information.xlsx")
df.describe()
df.head(50)

,solved,paths,n,m,2m/n,l,s,t,max_bicc,maxbi_occ,density,cycles,cycle_pcts,max_deg,l/short,calls,score,zdd
rocketfuel-k_o-10.col,True,4,11,12,2.18,10,6,8,4,36.36,21.82,2,18.18,4,2.0,22,NaN,NaN
stanford-feather-lastfm-social-48-49.col,True,1,49,48,1.96,49,17,33,2,4.08,4.08,0,0.0,5,4.08,13,NaN,NaN
rsndlib--36.col,False,NaN,100,154,3.08,36,5,86,88,88.0,3.11,55,55.0,7,4.5,NaN,20056,NaN
rocketfuel-k_o-99.col,False,NaN,240,404,3.37,99,1,230,130,54.17,1.41,165,68.75,31,7.07,NaN,28375,NaN
stanford-loc-Brightkite-9-10.col,True,1,10,9,1.8,10,1,2,2,20.0,20.0,0,0.0,7,3.33,4,NaN,NaN
stanford-musae-wiki_crocodile-1311-1200.col,False,NaN,1221,1311,2.15,1200,713,925,216,17.69,0.18,91,7.45,44,32.43,NaN,25036,NaN
matpower-case145-105.col,False,NaN,145,422,5.82,105,26,125,128,88.28,4.04,278,191.72,20,9.55,NaN,36905,NaN
rocketfuel-k_o-735.col,False,NaN,960,2821,5.88,735,285,447,678,70.62,0.61,1862,193.96,44,56.54,NaN,4932541,NaN
matpower-case_ACTIVSg500-127.col,True,227689183254166383,500,584,2.34,127,38,362,239,47.8,0.47,85,17.0,14,6.35,NaN,18483,2780859
stanford-Oregon-1-1557-543.col,False,NaN,1437,1557,2.17,543,178,662,161,11.2,0.15,121,8.42,203,33.94,NaN,41948,NaN
